In [1]:
import pandas as pd
import numpy as np
import torch
import os
import sys
from tqdm import tqdm, trange

sys.path.append("../../")
import biked_commons
from biked_commons.data_loading import data_loading

This builds the overall validation function

In [2]:
from bike_bench_validation_functions import bike_bench_validation_functions
from base_validation_function import construct_tensor_validator, construct_dataframe_validator

Read in the Bike Data

In [3]:
data_train = data_loading.load_bike_bench_train()
data_test = data_loading.load_bike_bench_test()
data = pd.concat([data_train, data_test], ignore_index=True)
display(data)

,CS textfield,BB textfield,Stack,Head angle,Head tube length textfield,Seat stay junction0,Seat tube length,Seat angle,DT Length,FORK0R,...,Handlebar style OHCLASS: 2,Stem kind OHCLASS: 0,Stem kind OHCLASS: 1,Stem kind OHCLASS: 2,Fork type OHCLASS: 0,Fork type OHCLASS: 1,Fork type OHCLASS: 2,Seat tube type OHCLASS: 0,Seat tube type OHCLASS: 1,Seat tube type OHCLASS: 2
0,420.00,25.0,536.36066,70.0,112.8,45.0,400.0,73.0,648.36316,45.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1,796.94,-3.0,363.41064,39.0,181.2,36.0,567.0,37.0,1243.38900,45.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,458.00,50.0,549.67426,63.5,159.9,45.0,460.0,76.0,742.93770,45.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
3,403.00,70.0,569.91583,73.0,134.0,258.0,516.8,79.0,627.59420,45.0,...,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
4,440.00,70.0,574.76190,74.0,125.2,45.0,579.6,74.0,651.83970,45.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4495,430.00,65.0,630.07196,72.5,194.1,60.0,550.3,73.5,639.51460,65.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
4496,385.00,47.0,527.72925,74.0,128.7,50.0,520.0,76.0,617.48610,30.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
4497,515.00,35.0,577.54860,65.0,117.0,11.0,550.2,66.0,662.99440,85.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
4498,435.00,20.0,527.21875,72.0,130.0,65.0,415.0,72.0,608.89920,45.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


This builds the overall validation function

In [4]:
dataframe_validator = construct_dataframe_validator(bike_bench_validation_functions)

Call the validation function and check the results. Remember, a score less or equal to 0 is valid. The more postive, the greater the magnitude of constraint violation. The more negative the further away from violating the constraint. A design is overall valid if all constraints are less or equal to 0

In [5]:
validity_scores = dataframe_validator(data)
display(validity_scores.head(10)) #design 6's saddle height is too small, for example

,Saddle height too small,Seat post too short,Head tube lower extension too great,Head tube length too great,Certain parameters must be positive,Chain stay should be greater than wheel radius,Chain stay should be greater than BB,Seat stay should be greater than wheel radius,Down tube must reach head tube,The pedal shouldn't intersect the front wheel,The crank shouldn't hit the ground when it is in its lower position,RGB value should be less than 255
0,-278.000000,-42.000000,-69.600006,-43.200005,0.0,-85.000000,-395.000000,-110.443512,-177.405121,-108.795349,-137.500000,-612.0
1,-36.200012,-283.799988,-134.199997,-66.199997,0.0,-454.040009,-799.940002,-150.122528,-964.433228,-1139.887939,-173.399994,-663.0
2,-240.000000,-230.000000,-69.899994,-41.599991,0.0,-85.500000,-408.000000,-127.807098,-255.819336,-245.684753,-150.000000,-306.0
3,-199.200012,-120.799988,-100.900002,-72.699997,0.0,-66.000000,-333.000000,-56.222290,-154.169495,-37.086548,-94.500000,-255.0
4,-98.400024,-191.599976,-81.500000,-50.500000,0.0,-103.000000,-370.000000,-191.612305,-155.420654,-46.163513,-94.500000,-714.0
5,-87.599976,-232.400024,-109.300003,-75.099998,0.0,-70.500000,-355.000000,-198.786499,-165.845276,-82.608948,-112.000000,-510.0
6,-95.799988,-224.200012,-99.600006,-67.700005,0.0,-68.000000,-335.000000,-165.000214,-141.723480,-25.680542,-94.500000,-612.0
7,564.500000,-854.500000,-86.900002,-57.900002,0.0,-52.339996,-339.339996,-206.046387,-169.684875,4.355347,-114.500000,0.0
8,-50.000000,-240.000000,-75.000000,-52.000000,0.0,-60.000000,-365.000000,-56.705353,-164.358459,-76.426147,-132.500000,-765.0
9,-15.000000,-275.000000,-60.900002,-40.000000,0.0,-56.799988,-356.799988,-63.294312,-156.791382,-74.689209,-127.500000,-306.0


In [6]:
#overall valid if all constraints are less or equal to 0

validity = np.all(validity_scores <= 0, axis=1)
print(validity[:10])
print(f"Fraction of valid designs: {np.sum(validity) / len(validity)}")

0     True
1     True
2     True
3     True
4     True
5     True
6     True
7    False
8     True
9     True
dtype: bool
Fraction of valid designs: 0.9108888888888889


If we want to calculate gradients, we need to use the pytorch version of the validator. 

In [7]:
COLUMN_NAMES = list(data.columns)
tensor_validator, validation_names = construct_tensor_validator(bike_bench_validation_functions, COLUMN_NAMES)

In [8]:
print(validation_names)

['Saddle height too small', 'Seat post too short', 'Head tube lower extension too great', 'Head tube length too great', 'Certain parameters must be positive', 'Chain stay should be greater than wheel radius', 'Chain stay should be greater than BB', 'Seat stay should be greater than wheel radius', 'Down tube must reach head tube', "The pedal shouldn't intersect the front wheel", "The crank shouldn't hit the ground when it is in its lower position", 'RGB value should be less than 255']


In [9]:
data_tensor = torch.tensor(data.values, dtype=torch.float32, requires_grad=True)
validity_scores_tens = tensor_validator(data_tensor)
print(validity_scores_tens) # should be the same as the dataframe version

loss = validity_scores_tens[validity_scores_tens > 0].sum() # we make our objective function equal to the sum of constraint violation
loss.backward()
gradients = data_tensor.grad
gradient_df = pd.DataFrame(gradients.numpy(), columns=COLUMN_NAMES, index = data.index)

tensor([[ -278.0000,   -42.0000,   -69.6000,  ...,  -108.7953,  -137.5000,
          -612.0000],
        [  -36.2000,  -283.8000,  -134.2000,  ..., -1139.8879,  -173.4000,
          -663.0000],
        [ -240.0000,  -230.0000,   -69.9000,  ...,  -245.6848,  -150.0000,
          -306.0000],
        ...,
        [  -94.8000,  -145.2000,   -85.3000,  ...,  -189.1634,  -137.5000,
          -341.0000],
        [  -95.0000,  -195.0000,   -70.0000,  ...,   -41.5312,  -142.5000,
          -306.0000],
        [ -118.0000,  -172.0000,  -188.1000,  ...,   -35.9692,   -92.5000,
             0.0000]], grad_fn=<CopySlices>)


The gradients tell us how to adjust the parameters to ideally fix the problem

In [10]:
display(gradient_df.head(10)) # we can see that there are some gradients in the seat tube length column, for example

,CS textfield,BB textfield,Stack,Head angle,Head tube length textfield,Seat stay junction0,Seat tube length,Seat angle,DT Length,FORK0R,...,Handlebar style OHCLASS: 2,Stem kind OHCLASS: 0,Stem kind OHCLASS: 1,Stem kind OHCLASS: 2,Fork type OHCLASS: 0,Fork type OHCLASS: 1,Fork type OHCLASS: 2,Seat tube type OHCLASS: 0,Seat tube type OHCLASS: 1,Seat tube type OHCLASS: 2
0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.165809,0.811091,7.364223,-0.785256,0.0,1.0,0.0,-1.460662,-1.028547,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Test gradient descent to see if validity improves

In [11]:
updated_data = data
step_size = 0.1
for steps in trange(1000):
    #calculate gradients
    data_tensor = torch.tensor(updated_data.values, dtype=torch.float32, requires_grad=True)
    validity_scores_tens = tensor_validator(data_tensor)
    loss = validity_scores_tens[validity_scores_tens > 0].sum()
    loss.backward()
    gradients = data_tensor.grad
    gradients = gradients.numpy()
    updated_data = updated_data - step_size * gradients

100%|██████████| 1000/1000 [00:15<00:00, 62.59it/s]


In [12]:
#check validity of updated data
validity_scores = dataframe_validator(updated_data)
validity = np.all(validity_scores <= 0, axis=1)
print(validity.mean())

0.994


In [13]:
display(validity_scores.head(10))


,Saddle height too small,Seat post too short,Head tube lower extension too great,Head tube length too great,Certain parameters must be positive,Chain stay should be greater than wheel radius,Chain stay should be greater than BB,Seat stay should be greater than wheel radius,Down tube must reach head tube,The pedal shouldn't intersect the front wheel,The crank shouldn't hit the ground when it is in its lower position,RGB value should be less than 255
0,-278.000000,-42.000000,-69.600006,-43.200005,0.0,-85.000000,-395.000000,-110.443512,-177.405121,-108.795349,-137.500000,-612.0
1,-36.200012,-283.799988,-134.199997,-66.199997,0.0,-454.040009,-799.940002,-150.122528,-964.433228,-1139.887939,-173.399994,-663.0
2,-240.000000,-230.000000,-69.899994,-41.599991,0.0,-85.500000,-408.000000,-127.807098,-255.819336,-245.684753,-150.000000,-306.0
3,-199.200012,-120.799988,-100.900002,-72.699997,0.0,-66.000000,-333.000000,-56.222290,-154.169495,-37.086548,-94.500000,-255.0
4,-98.400024,-191.599976,-81.500000,-50.500000,0.0,-103.000000,-370.000000,-191.612305,-155.420654,-46.163513,-94.500000,-714.0
5,-87.599976,-232.400024,-109.300003,-75.099998,0.0,-70.500000,-355.000000,-198.786499,-165.845276,-82.608948,-112.000000,-510.0
6,-95.799988,-224.200012,-99.600006,-67.700005,0.0,-68.000000,-335.000000,-165.000214,-141.723480,-25.680542,-94.500000,-612.0
7,364.500000,-654.500000,-87.057053,-58.057060,0.0,-52.339996,-339.356567,-136.106720,-169.776978,-1.618958,-114.516586,0.0
8,-50.000000,-240.000000,-75.000000,-52.000000,0.0,-60.000000,-365.000000,-56.705353,-164.358459,-76.426147,-132.500000,-765.0
9,-15.000000,-275.000000,-60.900002,-40.000000,0.0,-56.799988,-356.799988,-63.294312,-156.791382,-74.689209,-127.500000,-306.0


In [14]:
margin = 0.05
parameter_ranges = np.max(data.values, axis=0) - np.min(data.values, axis=0)
lower_bounds = np.min(data.values, axis=0)-margin*parameter_ranges
upper_bounds = np.max(data.values, axis=0)+margin*parameter_ranges

#randomly sample a design
random_design = np.random.uniform(lower_bounds, upper_bounds)

#check validity of random design

validity_scores = dataframe_validator(pd.DataFrame(random_design.reshape(1, -1), columns=COLUMN_NAMES))
display(validity_scores) # should be all less than or equal to 0


,Saddle height too small,Seat post too short,Head tube lower extension too great,Head tube length too great,Certain parameters must be positive,Chain stay should be greater than wheel radius,Chain stay should be greater than BB,Seat stay should be greater than wheel radius,Down tube must reach head tube,The pedal shouldn't intersect the front wheel,The crank shouldn't hit the ground when it is in its lower position,RGB value should be less than 255
0,35.682434,-663.143616,-1067.778076,-908.504028,294.864807,-1840.256348,-2009.275269,-970.148376,-965.886597,343.587097,3.481049,-579.408142
